In [1]:
import os
os.environ["DATASET_PERIOD"] = "100"
os.environ["PMODE"] = "pmode_0003" # same as jetson orin
os.environ["STREVAL_TRAIN"] = "1"
os.environ["FINE_GRAINED_EVAL"] = "1"

os.chdir("/root/shared/Anytime-Lidar/tools")

import _init_path
import datetime
import time
import json
import math
from pathlib import Path

import torch
import gc
import sys
import pickle
import numpy as np
from alive_progress import alive_bar

from eval_utils import eval_utils
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

import matplotlib.pyplot as plt

def get_dataset(cfg):
    log_file = ('./tmp_results/log_eval_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=0)
    test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, batch_size=1,
        dist=False, workers=0, logger=logger, training=False
    )

    return logger, test_set, test_loader, sampler

In [ ]:
cfg_file = "./cfgs/nuscenes_models/pillar01_015_02_024_03_valor.yaml"
cfg_from_yaml_file(cfg_file, cfg)

set_cfgs = ['MODEL.METHOD', '0', 'MODEL.DEADLINE_SEC', '100.0', 'MODEL.DENSE_HEAD.NAME', 'CenterHeadInf', 'OPTIMIZATION.BATCH_SIZE_PER_GPU', '1']
cfg_from_list(set_cfgs, cfg)
logger, test_set, test_loader, sampler = get_dataset(cfg)
print(f'Loaded dataset with {len(test_set)} samples')

def calc_tail_ms(cur_time_point_ms, data_period_ms):
    return cur_time_point_ms - math.floor(cur_time_point_ms / data_period_ms) * data_period_ms

ckpt_file="../output/nuscenes_models/pillar01_015_02_024_03_valor/default/ckpt/checkpoint_epoch_30.pth"

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
model.load_params_from_file(filename=ckpt_file, logger=logger, to_cpu=False)
# model.pre_hook_handle.remove()
# model.post_hook_handle.remove()
model.eval() # should be run with @torch.no_grad
model.cuda()

data_period_ms = 50
num_samples = len(test_set)

last_sample_idx = 0
cur_sample_idx = 0
sim_cur_time_ms = 0.
last_exec_time_ms = 100.
pred_dicts_arr = []
output_times_ms = []
processed_inds = set()

model.calibrate()
model.res_idx = 2

with alive_bar(num_samples, force_tty=True, max_cols=160, manual=True) as bar:
    while cur_sample_idx < num_samples:
        with torch.no_grad():
            #batch_dict = model.dataset.collate_batch([data_dict])
            #load_data_to_gpu(batch_dict)
            pred_dicts, ret_dict = model([cur_sample_idx])

        # Predict the execution time as if the DNN were to be executed on target platform
        batch_dict = model.latest_batch_dict
        last_exec_time_ms = model.calibrators[model.res_idx].pred_exec_time_ms(
           batch_dict['points'].size(0),
           np.array([batch_dict['bb3d_num_voxels']]),
           batch_dict['x_lims'][1] - batch_dict['x_lims'][0])

        sim_cur_time_ms += last_exec_time_ms
        pred_dicts_arr.append(pred_dicts) # last one is output time
        output_times_ms.append(sim_cur_time_ms)
        processed_inds.add(cur_sample_idx)

        #Dynamic scheduling
        cur_tail = calc_tail_ms(sim_cur_time_ms, data_period_ms)
        pred_finish_time = sim_cur_time_ms + last_exec_time_ms #NOTE I can also use mean exec time
        next_tail = calc_tail_ms(pred_finish_time, data_period_ms)
        if next_tail < cur_tail:
            # Sleep, extra 1 ms is added to make sure sleet time is enough
            sim_cur_time_ms += data_period_ms - cur_tail + 1

        cur_sample_idx = int(sim_cur_time_ms / data_period_ms)
        if cur_sample_idx in processed_inds:
            print(f'ERROR, trying to process already processed sample {cur_sample_idx}')

        bar(cur_sample_idx / num_samples)
        last_sample_idx = cur_sample_idx

output_times_ms = np.array(output_times_ms)
model.print_time_stats()

2024-10-29 17:53:31,305   INFO  Loading NuScenes dataset
2024-10-29 17:53:33,855   INFO  Total samples for NuScenes dataset: 27332


Loaded dataset with 27332 samples
Default deadline is: 100.0


2024-10-29 17:53:34,845   INFO  ==> Loading parameters from checkpoint ../output/nuscenes_models/pillar01_015_02_024_03_valor/default/ckpt/checkpoint_epoch_30.pth to GPU
2024-10-29 17:53:36,199   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+31546c7+py52e9ef4
2024-10-29 17:53:36,236   INFO  ==> Done (loaded 2106/2106)


End to end execution time stats (ms):
Min	1Perc	5Perc	Mean	95Perc	99Perc	Max
72.12	74.36	80.09	114.91	146.66	155.11	159.81
Calibrating resolution 0
Resolution idx: 0 Input: x_conv4 torch.Size([1, 256, 144, 144])
Trying to load trt engine at ./deploy_files/trt_engines/pmode_0003/pillarnet01_valo_dense_convs_res0.engine
TensorRT engine ./deploy_files/trt_engines/pmode_0003/pillarnet01_valo_dense_convs_res0.engine successfully loaded.
Optimization took 1.6199960708618164 seconds.
Num params: 15325638
Num params trainable: 15325638
Detector3D calibration done
End to end execution time stats (ms):
Min	1Perc	5Perc	Mean	95Perc	99Perc	Max
55.12	56.88	62.96	76.96	91.30	93.92	104.57
Calibrating resolution 1
Resolution idx: 1 Input: x_conv4 torch.Size([1, 256, 96, 96])
Trying to load trt engine at ./deploy_files/trt_engines/pmode_0003/pillarnet01_valo_dense_convs_res1.engine
TensorRT engine ./deploy_files/trt_engines/pmode_0003/pillarnet01_valo_dense_convs_res1.engine successfully loaded.
Optimiz

In [ ]:
# det_annos = []
# for bd, pred_dicts, outp_time_point in pred_tuples:
#     annos = dataset.generate_prediction_dicts(
#         bd, pred_dicts, class_names, output_path=op
#     )
#     det_annos += annos

#We have:
# pred_dicts_arr
# output_times_ms
# processed_inds

def get_streaming_eval_samples(num_ds_elems : int, period_ms : int, output_times_ms : np.ndarray, pred_dicts_arr):
    # Streaming eval
    #Now do manual sampling base on time

    #times_ns should be output_times_ms
    sampled_objects = []
    for i in range(num_ds_elems):
        sample_time_ms = i*period_ms
        aft = output_times_ms > sample_time_ms
        if aft[0] == True:
            sampled_objects.append(None) # Should be an empty pred dicts arr
#             print('0', end=' ')
        elif aft[-1] == False:
            sampled_objects.append(pred_dicts_arr[-1])
#             print('-1', end=' ')
        else:
            sample_idx = np.argmax(aft) - 1
            sampled_objects.append(pred_dicts_arr[sample_idx])
#             print(sample_idx, end=' ')
#     print()

    return sampled_objects

def do_eval(sampled_objects, dataset):
    #Convert them to openpcdet format

    det_annos = []
    num_ds_elems = len(dataset)
    for i in range(num_ds_elems):
        data_dict = dataset.get_metadata_dict(i)
        for k, v in data_dict.items():
            data_dict[k] = [v] # make it a batch dict
        pred_dicts = sampled_objects[i]

        if pred_dicts is None:
            pred_dicts = [{
                'pred_boxes': torch.empty((0, 9)),
                'pred_scores': torch.empty(0),
                'pred_labels': torch.empty(0, dtype=torch.long)
            }]
        data_dict['final_box_dicts'] = pred_dicts
        det_annos += dataset.generate_prediction_dicts(
            data_dict, data_dict['final_box_dicts'], dataset.class_names, output_path=None
        )

    calib_dl = model.res_idx
    eval_d = {
        'cfg': cfg,
        'det_annos': det_annos,
        'annos_in_glob_coords': False,
        'calib_deadline_ms': model.res_idx}

    #nusc_annos = {} # not needed but keep it anyway
    result_str, result_dict = dataset.evaluation(
        det_annos, dataset.class_names,
        eval_metric=cfg.MODEL.POST_PROCESSING.EVAL_METRIC,
        output_path='./tmp_results',
        boxes_in_global_coords=False,
    )

    print(result_str)
    eval_d['result_str'] = result_str

    with open(f'eval_data_{calib_dl}ms.pkl', 'wb') as f:
        pickle.dump(eval_d, f)

sampled_objects = get_streaming_eval_samples(len(test_set), data_period_ms, output_times_ms, pred_dicts_arr)
print(f'Sampled {len(sampled_objects)} objects')
do_eval(sampled_objects, model.dataset)